In [ ]:
import sys
import numpy as np
import pandas as pd
import usaddress
import os
import requests
import datetime
import re
import time
from bs4 import BeautifulSoup

In [ ]:
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
headers = {"user-agent" : USER_AGENT}

In [ ]:
# function to parse address from webpage
def url_postal_adddress(url):
#     USER_AGENT = "Mozilla/5.0 (Linux; <Android Version>; <Build Tag etc.>) AppleWebKit/<WebKit Rev>(KHTML, like Gecko) Chrome/<Chrome Rev> Safari/<WebKit Rev>"
    USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
    headers = {"user-agent" : USER_AGENT}
    
    RE_address=re.compile(r"""(
                \d{1,5}[ ]    #street number
                (?:E\.[ ]|S\.[ ]|W\.[ ]|N\.[ ])?    #east south west north prefix
                [a-zA-Z0-9 ]{1,20}    #street name
                (?:street|st|avenue|ave|road|rd|highway|hwy|square|sq|trail|trl|drive|dr|court|ct|park|parkway|pkwy|circle|cir|boulevard|blvd|walk)\W?(?=\s)\s*    #street surffix
                (?:\w{10})?
                [a-zA-Z\']{1,14}(?:\s[a-zA-Z\']{1,14}){0,1} #city
                (?:,\s|\b)?(?:AL|AK|AR|AZ|CA|CO|CT|DC|DE|FL|GA|HI|IA|ID|IL|IN|KS|KY|LA|MA|MD|ME|MI|MN|MO|MS|MT|NC|ND|NE|NH|NJ|NM|NV|NY|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VA|VT|WA|WI|WV|WY) #state
                (?:,\s|\s)?\b\d{5}(?:[-\s]\d{4})?\b    #zipcode
                 )""", re.IGNORECASE | re.VERBOSE)
    
    address_parsed_list=pd.DataFrame(columns=('AddressRaw',
    'AddressNumberPrefix',
    'AddressNumber',
    'AddressNumberSuffix',
    'StreetNamePreModifier',
    'StreetNamePreDirectional',
    'StreetNamePreType',
    'StreetName',
    'StreetNamePostType',
    'StreetNamePostDirectional',
    'SubaddressType',
    'SubaddressIdentifier',
    'BuildingName',
    'OccupancyType',
    'OccupancyIdentifier',
    'CornerOf',
    'LandmarkName',
    'PlaceName',
    'StateName',
    'ZipCode',
    'USPSBoxType',
    'USPSBoxID',
    'USPSBoxGroupType',
    'USPSBoxGroupID',
    'IntersectionSeparator',
    'Recipient'))
    
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.content, "html.parser")
    address_list=re.findall(RE_address,soup.prettify())
    address_list=list(set(address_list))
    for address in address_list:
        address_parsed=np.flip(usaddress.parse(address),1).T
        address_df = pd.DataFrame(data=address_parsed,columns=address_parsed[0,:])
        address_df = address_df[1:]
        address_df['AddressRaw']=address
        address_df = address_df.groupby(address_df.columns, axis=1).apply(lambda x: x.apply(lambda y: ' '.join([l for l in y if l is not None]), axis=1))
        address_parsed_list=address_parsed_list.append(address_df)
    return address_parsed_list

In [ ]:
university_list=pd.read_csv("universities.csv",encoding='latin1')['University_Name']
keyword_list=['dormitory','dormitory address', 'residence hall', 'residential hall', 'residence hall address', 'residential hall address','graduate student address']

In [ ]:
# for each university and each keyword, search and parse the address from google map result page
university_address_all = pd.DataFrame()
URL_address_missed = pd.DataFrame()
i=0
for keyword in keyword_list:
    for university in university_list:
        try:
            search_word='+'.join((university+'+'+keyword).split())
            URL = f"https://www.google.com/maps/search/{search_word}".format(search_word=search_word)
            address_df=url_postal_adddress(URL)
            google_map_driver_df=pd.DataFrame(columns=('University','Keyword','join_key'))
            google_map_driver_df=google_map_driver_df.append(pd.Series([university,keyword,1], index=['University','Keyword','join_key']),ignore_index=True)
            address_df['join_key']=1
            google_map_address_df=pd.merge(google_map_driver_df,address_df).drop(columns=['join_key'])
            if google_map_address_df.size>0:
                if university_address_all.size > 0:
                    university_address_all=university_address_all.append(google_map_address_df)
                else:
                    university_address_all=google_map_address_df
                university_address_all.to_csv('university_address.csv')
                print( i ,university ,'|', keyword ,'|', google_map_address_df.index.size-1)
            else:
                if URL_address_missed.size>0:
                    URL_address_missed=URL_address_missed.append(google_map_driver_df)
                else:
                    URL_address_missed=google_map_driver_df
                URL_address_missed.to_csv('university_address_missed.csv')
                
                print(i ,'|', university ,'|', keyword ,'|', 'no record')
            i=i+1
            time.sleep(1)
        except:
            if URL_address_missed.size>0:
                URL_address_missed=URL_address_missed.append(google_map_driver_df)
            else:
                URL_address_missed=google_map_driver_df
            URL_address_missed.to_csv('university_address_missed.csv')
            time.sleep(2)
            print('error')